In [1]:
!pwd
from google.colab import drive
drive.mount('/content/drive')

/content
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# unzip
import zipfile, os, shutil

dataset = '/content/drive/MyDrive/datasets/cifake.zip'
data_path = '/content/drive/MyDrive/cifake_local'
data_file = os.path.join(data_path, 'cifake.zip')

if not os.path.exists(data_path):
  os.makedirs(data_path)

# copy zip file
shutil.copy(dataset, data_file)
  
with zipfile.ZipFile(data_file, 'r') as file: 
  file.extractall(data_path)

KeyboardInterrupt: ignored

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

In [4]:

dataset_dir = '/content/drive/MyDrive/cifake_small/cifake_small' # For Kaggle notebooks. If you run locally, point this line to the CIFAKE directory
print("Loading dataset from: " + dataset_dir)

Loading dataset from: /content/drive/MyDrive/cifake_small/cifake_small


In [5]:
# Import all of the data using dataset from directory
# If there isn't enough RAM available, consider using Tensorflow Datasets 

import tensorflow as tf

img_height = 32 # The dataset is all 32px but this is here just to make sure
img_width = 32
batch_size = 32

# Load the training data
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir + "/train",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=None)

# Load the validation data
val_ds = tf.keras.utils.image_dataset_from_directory(
  dataset_dir + "/valid",
  seed=123,
  image_size=(img_height, img_width),
    batch_size=None)

# Load the validation data
test_ds = tf.keras.utils.image_dataset_from_directory(
  dataset_dir + "/test",
  seed=123,
  image_size=(img_height, img_width),
    batch_size=None)

# Quick sanity check to make sure it's all loaded properly
print("Training Classes:")
class_names = train_ds.class_names
print(class_names)


print("Training Classes:")
class_names = val_ds.class_names
print(class_names)


print("Testing Classes:")
class_names = test_ds.class_names
print(class_names)

Found 1000 files belonging to 2 classes.
Found 502 files belonging to 2 classes.
Found 498 files belonging to 2 classes.
Training Classes:
['fake', 'real']
Training Classes:
['fake', 'real']
Testing Classes:
['fake', 'real']


In [ ]:
# for img_batch, label_batch in train_ds.take(1) : 
#     print(img_batch.shape)
#     print(label_batch.numpy()) 

In [6]:
def normalize_and_resize(image, label):
    image = tf.cast(image, tf.float32)
    image = image / 255.
    image = tf.image.resize(image, (32, 32))
 
    return image, label

In [7]:
train_batches = train_ds.map(normalize_and_resize).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
validation_batches = val_ds.map(normalize_and_resize).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
test_batches = test_ds.map(normalize_and_resize).batch(1)
print(train_batches)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [8]:
def build_model():
    base_model = tf.keras.applications.VGG16(input_shape= (32,32, 3),
                                             weights='imagenet',
                                             include_top=False)
    
    # add a GAP layer
    output = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
 
    # output has two neurons for the 2 classes(dogs and cats)
    output = tf.keras.layers.Dense(2, activation='softmax')(output)
 
    # set the inputs and outputs of the model
    model = tf.keras.Model(base_model.input, output)
 
    for layer in base_model.layers[:-4]:
        layer.trainable = False
    
    model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['acc'])
    model.summary()
 
    return model
    
model = build_model()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [9]:
EPOCHS = 10
model.fit(train_batches, epochs=EPOCHS, validation_data=validation_batches)

Epoch 1/10
32/32 [==============================] - 9s 60ms/step - loss: 3.1798 - acc: 0.5480 - val_loss: 0.6740 - val_acc: 0.5857
Epoch 2/10
32/32 [==============================] - 2s 36ms/step - loss: 0.5820 - acc: 0.7150 - val_loss: 0.7936 - val_acc: 0.6474
Epoch 3/10
32/32 [==============================] - 2s 35ms/step - loss: 0.5078 - acc: 0.7770 - val_loss: 0.4106 - val_acc: 0.8267
Epoch 4/10
32/32 [==============================] - 2s 35ms/step - loss: 0.4511 - acc: 0.8200 - val_loss: 0.4539 - val_acc: 0.7888
Epoch 5/10
32/32 [==============================] - 2s 31ms/step - loss: 0.3931 - acc: 0.8250 - val_loss: 0.5262 - val_acc: 0.7410
Epoch 6/10
32/32 [==============================] - 3s 58ms/step - loss: 0.3140 - acc: 0.8650 - val_loss: 0.3644 - val_acc: 0.8386
Epoch 7/10
32/32 [==============================] - 2s 34ms/step - loss: 0.2654 - acc: 0.8970 - val_loss: 0.8980 - val_acc: 0.7131
Epoch 8/10
32/32 [==============================] - 2s 31ms/step - loss: 0.2851 - a

In [10]:
EPOCHS = 10
model.fit(train_batches, epochs=EPOCHS, validation_data=validation_batches)

Epoch 1/10
32/32 [==============================] - 5s 99ms/step - loss: 0.1955 - acc: 0.9560 - val_loss: 0.8601 - val_acc: 0.7908
Epoch 2/10
32/32 [==============================] - 3s 62ms/step - loss: 0.0855 - acc: 0.9660 - val_loss: 0.7518 - val_acc: 0.8705
Epoch 3/10
32/32 [==============================] - 3s 49ms/step - loss: 0.0910 - acc: 0.9700 - val_loss: 0.5399 - val_acc: 0.8665
Epoch 4/10
32/32 [==============================] - 3s 56ms/step - loss: 0.1367 - acc: 0.9660 - val_loss: 0.4994 - val_acc: 0.8506
Epoch 5/10
32/32 [==============================] - 3s 63ms/step - loss: 0.0155 - acc: 0.9950 - val_loss: 1.0652 - val_acc: 0.8466
Epoch 6/10
32/32 [==============================] - 2s 36ms/step - loss: 1.9112e-04 - acc: 1.0000 - val_loss: 1.0218 - val_acc: 0.8625
Epoch 7/10
32/32 [==============================] - 2s 35ms/step - loss: 3.0941e-05 - acc: 1.0000 - val_loss: 1.1057 - val_acc: 0.8586
Epoch 8/10
32/32 [==============================] - 2s 32ms/step - loss: 1.

In [12]:
cam_model = tf.keras.Model(model.input, outputs=(model.layers[-3].output, model.layers[-1].output))
cam_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0   

In [13]:
gap_weights = model.layers[-1].get_weights()[0]
gap_weights.shape

(512, 2)

In [17]:
def show_cam(image_value, features, results, label):
    '''
    Displays the class activation map of an image
 
    Args:
        image_value (tensor) -- preprocessed input image with size 224 x 224
        features (array) -- features of the image, shape (1, 7, 7, 512)
        results (array) -- output of the sigmoid layer
    '''
    features_for_img = features[0]
    prediction = results[0]
 
    class_activation_weigths = gap_weights[:,label]
    class_activation_features = sp.ndimage.zoom(features_for_img, (32/7, 32/7, 1), order=2)
    cam_output = np.dot(class_activation_features, class_activation_weigths)
    cam_output = tf.reshape(cam_output, (224,224 ))
 
    # visualize the results
    print(f'sigmoid output: {results}')
    print(f"prediction: {'real' if tf.argmax(results[0]) else 'fake'}")
    plt.figure(figsize=(8,8))
    plt.imshow(cam_output, cmap='jet', alpha=0.5)
    plt.imshow(tf.squeeze(image_value), alpha=0.5)
    plt.show()

In [18]:
# preprocess the test images
augmented_test_data = val_ds.map(normalize_and_resize)
test_batches = augmented_test_data.batch(1)

In [ ]:
for img, lbl in test_batches.take(10):
    print(f"ground truth: {'real' if lbl else 'fake'}")
    features,results = cam_model.predict(img)
    show_cam(img, features, results, lbl)
